In [7]:
%matplotlib inline
from collections import defaultdict, Counter
import copy
import pandas as pd
import sys
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import scipy
import statsmodels.api as sm
import time

# Models

In [25]:
class SimpleMultivariateHawkesProcess:
    _ndim = None # no. of dimensions
    _lambda = None # ndim x 1 vector
    _alpha = None # ndim x ndim matrix
    _beta = None # ndim x ndim matrix
    _observation_window = None
    _EPS = 10**-9 # numbers below this are set to 0

    def __init__(self, ndim=None, lam=None, alpha=None, beta=None, observation_window=None):
        self._ndim = ndim
        self._lambda = lam
        self._alpha = alpha
        self._beta = beta
        self._observation_window = observation_window
    
    def fit(self, ndim, alpha, data_epochs, data_dims, observation_window, niter=1000):
        print 'Fitting...'
        print '\tPrecomputing A(i), B(i)...'
        #A, B = self._par_precompute(ndim, alpha, data_epochs, data_dims, observation_window)
        A, B = self._precompute(ndim, alpha, data_epochs, data_dims, observation_window)
                
        print '\tMinimizing negative loglikelihood...'
        alpha_ravel = alpha.ravel()
        x0 = np.random.uniform(low=10**-5.0, high=alpha_ravel[0] - self._EPS, size=(ndim * (ndim + 1)))
        bounds = [(self._EPS, None) for i in range(ndim)] +\
                 [(self._EPS, None) for i in range(ndim * ndim)] 
                 #[(self._EPS, None) for i in range(ndim * ndim)]
                            
        x, f, d = scipy.optimize.fmin_l_bfgs_b(self._seq_negloglikelihood,
                                               x0=x0, args=(ndim, alpha, observation_window, A, B),
                                               factr=10.0**7.0, iprint=99, pgtol=10**-5, approx_grad=False,
                                               callback=None, disp=99,
                                               bounds=bounds, maxiter=niter, maxfun=niter)

        return x, f, d
    
    @staticmethod
    def _precompute(ndim, alpha, data_epochs, data_dims, observation_window):
        T0, T = observation_window
        nsamples = len(data_epochs)

        A = []
        B = [np.zeros(shape=(ndim, ndim), dtype=np.float64) for k in range(nsamples)]
        for k in range(nsamples):
            # memoize locations of timestamps for each dimension for this sample (2s for 18K samples)
            epochs_per_dimension = []
            for i in range(ndim):
                idx_i = data_dims[k] == i
                epochs_i = data_epochs[k][idx_i]
                epochs_per_dimension.append(epochs_i)
            
            A_k = [np.zeros(shape=(ndim, len(epochs_per_dimension[m])), dtype=np.float64)
                   for m in range(ndim)]
            #B_mn = np.zeros(shape=(ndim, ndim), dtype=np.float64)
            for m in range(ndim):
                epochs_m = epochs_per_dimension[m]
                if len(epochs_m) == 0:
                    continue # no transactions on dimension m
                #A_m = np.zeros(shape=(ndim, len(epochs_m)), dtype=np.float64)
                for n in range(ndim):
                    epochs_n = epochs_per_dimension[n]
                    if len(epochs_n) == 0: # no transactions on dimension n
                        continue
                    len_n = len(epochs_n)
                    A_mn = A_k[m][n] # influence of dimension n on each timestamp of dimension m

                    j = 0
                    if (m != n): # mutual excitation    
                        
                        while j < len_n and epochs_n[j] < epochs_m[0]:
                            j += 1
                            
                        #A_mn[0] = np.sum(np.exp(-alpha[m,n] * (epochs_m[0] - epochs_n[epochs_n < epochs_m[0]]) / 1000.0))
                        A_mn[0] = np.sum(np.exp(-alpha[m,n] * (epochs_m[0] - epochs_n[:j]) / 1000.0))

                        for i in range(1, len(epochs_m)):
                            prev_j = j
                            while j < len_n and epochs_n[j] < epochs_m[i]:
                                j += 1
                            A_mn[i] = np.exp(-alpha[m,n] * (epochs_m[i] - epochs_m[i-1]) / 1000.0) * A_mn[i-1]
                            #A_mn[i] += np.sum(np.exp(-alpha[m,n] *
                            #                         (epochs_m[i] - epochs_n[(epochs_n >= epochs_m[i-1]) &
                            #                                                 (epochs_n < epochs_m[i])]) / 1000.0))
                            A_mn[i] += np.sum(np.exp(-alpha[m,n] *
                                                     (epochs_m[i] - epochs_n[prev_j:j]) / 1000.0))
                        
                        B[k][m,n] = np.sum(1 - np.exp(-alpha[m,n] * (T - epochs_n) / 1000.0))
                    else: # dim_m = dim_n, self-excitation
                        for i in range(1, len(epochs_m)):
                            A_mn[i] = np.exp(-alpha[m,n] * (epochs_m[i] - epochs_m[i-1]) / 1000.0) * (1 + A_mn[i-1])
                        B[k][m,n] = np.sum(1 - np.exp(-alpha[m,n] * (T - epochs_m) / 1000.0))
                
                #A_k.append(A_m)
            
            A.append(A_k)
            #B[k] = B_mn

        return A, B
    
    def _seq_negloglikelihood(self, params, ndim, alpha, observation_window, A, B):
        nsamples = len(A)
        nll = 0.0
        grad = np.zeros(shape=((ndim + 1) * ndim), dtype=np.float64) # first row is grad of lambda
        for k in range(nsamples):
            sample_nll, sample_grad = self._negloglikelihood(params, ndim, alpha, observation_window, A[k], B[k])
            nll += sample_nll
            grad += sample_grad
        nll /= nsamples
        grad /= nsamples
        return nll, grad
    
    @staticmethod
    def _negloglikelihood(params, ndim, alpha, observation_window, A, B):
        lam = params[:ndim] # first ndim elements
        beta = params[ndim:].reshape((ndim, ndim)) # next ndim * ndim elements
        T0, T = observation_window
        T_length = (T - T0)/1000.0 # scale down
        
        loglikelihood = 0.0
        grad_lambda = np.zeros(shape=(ndim), dtype=np.float64)
        grad_beta = np.zeros(shape=(ndim, ndim), dtype=np.float64)

        for m in range(ndim):
            Z = lam[m] + np.sum(beta[m,:][:,None] * A[m], axis=0)
            loglikelihood += np.sum(np.log(Z)) - np.sum(beta[m,:] * B[m,:] / alpha[m,:]) - lam[m] * T_length
            grad_lambda[m] = np.sum(1.0/Z) - T_length
            grad_beta[m] = np.sum(A[m]/Z, axis=1) - B[m,:]/alpha[m,:]
            
            #print np.sum(np.log(Z)) - np.sum(beta[m,:] * B[m,:] / alpha[m,:]) - lam[m] * T_length \
            #   - len(A[m][0]) * np.log(1000) # compare with Nan
            #print 1000.0 * np.sum(1.0/Z) - T_length * 1000.0 # compare with Nan
            #print 1000.0 * grad_beta[m] # compare with nan
        
        gradient = np.concatenate((grad_lambda, grad_beta.ravel()))
        return -loglikelihood, -gradient

    def check_gradient(self, ndim, alpha, data_epochs, data_dims, observation_window, delta=10**-9, niter=50):
        print 'Checking gradient...'
        print '\tPrecomputing A, B...'
        A, B = self._precompute(ndim, alpha, data_epochs, data_dims, observation_window)
        
        nsamples = len(data_epochs)
        for i in range(niter):
            row_idx = np.random.randint(0, nsamples)
            a = A[row_idx]
            b = B[row_idx]
            x0 = np.array([np.random.uniform(10**-3, 10**3), np.random.uniform(10**-3, 10**3),  # lambda
                           np.random.uniform(10**-3, 10**3), np.random.uniform(10**-3, 10**3),  # beta
                           np.random.uniform(10**-3, 10**3), np.random.uniform(10**-3, 10**3)]) # beta

            nll, grad = self._negloglikelihood(x0, ndim, alpha, observation_window, a, b)
            param_idx = np.random.randint(len(x0))
            x0[param_idx] += delta
            new_nll, new_grad = self._negloglikelihood(x0, ndim, alpha, observation_window, a, b)
            num_grad = (new_nll - nll)/delta            
            if abs((grad[param_idx] - num_grad)/num_grad)  > 0.01:
                print 'Gradient checking failed for parameter:', param_idx
                print 'Analytical gradient:', grad[param_idx],
                print 'Numerical gradient:', num_grad
        print '\tGradient checking complete.'
        
    def simulate(self, t0, prev_t, prev_d, horizon_duration):
        simulated_iats = []
        simulated_dims = []
        prev_t_copy = copy.copy(list(prev_t))
        prev_d_copy = copy.copy(list(prev_d))
        prev_t_array = np.array(prev_t_copy, dtype=np.float64)
        prev_d_array = np.array(prev_d_copy, dtype=np.int)
        cumulative_iat = 0.0
        while cumulative_iat < horizon_duration:
            upper_bound = np.sum(self.intensity(t0 + cumulative_iat, (prev_t_array, prev_d_array), include_t0=True))
            s = 1000.0 * np.random.exponential(1.0/upper_bound) # params are in 1/1000s units            
                
            cumulative_iat += s # move ahead in time (may accept or reject)
            if cumulative_iat >= horizon_duration:
                break

            lambda_t_dim = self.intensity(t0 + cumulative_iat, (prev_t_array, prev_d_array))
            lambda_t = np.sum(lambda_t_dim)

            u = np.random.uniform(0.0, 1.0)
            if u <= lambda_t/upper_bound: # accept
                prev_t_copy.append(t0 + cumulative_iat)
                prev_t_array = np.array(prev_t_copy, dtype=np.float64)
                simulated_iats.append(cumulative_iat)
                
                cumprob = np.cumsum(lambda_t_dim/lambda_t)
                u2 = np.random.uniform(0.0, 1.0)
                dim = np.searchsorted(cumprob, u2)
                
                prev_d_copy.append(dim)
                prev_d_array = np.array(prev_d_copy, dtype=np.int)
                simulated_dims.append(dim)

        return simulated_iats, simulated_dims
    
    def predict_dist(self, t0, prev_t, prev_d, niter=1000):
        #step = 3600.0 - (t0 % 3600)
        simulated_iats = np.zeros(shape=(niter), dtype=np.float64)
        simulated_dims = np.zeros(shape=(niter), dtype=np.int)
        for i in range(niter):
            cumulative_iat = 0.0
            while True:
                upper_bound = np.sum(self.intensity(t0 + cumulative_iat, (prev_t, prev_d), include_t0=True))
                s = 1000.0 * np.random.exponential(1.0/upper_bound) # params are in 1/1000s units

                #print 'Candidate next IAT:', t + s
                #if s >= step: # upper bound no longer valid
                #    t += step
                #    step = 3600.0 # one hour in seconds
                #    #print '\tLarger than step, moved to:', t
                #    continue
                
                cumulative_iat += s # move ahead in time (may accept or reject)
                #step -= s
                lambda_t_dim = self.intensity(t0 + cumulative_iat, (prev_t, prev_d))
                lambda_t = np.sum(lambda_t_dim)
                u = np.random.uniform(0.0, 1.0)
                
                if u <= lambda_t/upper_bound: # accept
                    simulated_iats[i] = cumulative_iat
                    if self._ndim > 1:
                        simulated_dims[i] = np.random.choice(range(self._ndim), p=lambda_t_dim/lambda_t)
                    break
    
        return simulated_iats, simulated_dims

    def intensity(self, t0, prev_e, include_t0=False):
        prev_t, prev_d = prev_e
        if not include_t0:
            prev_t_ = prev_t[prev_t < t0]
            prev_d_ = prev_d[prev_t < t0]
        else:
            prev_t_ = prev_t[prev_t <= t0]
            prev_d_ = prev_d[prev_t <= t0]
        t0_minus_ti = (t0 - prev_t_) / 1000.0
        return self._lambda + np.sum(self._beta[:, prev_d_] * np.exp(-self._alpha[:, prev_d_] * t0_minus_ti), axis=1)
    
    def intensity_integral(self, start_t, end_t, data_epochs, data_dims):
        # memoize locations of timestamps for each dimension for this sample (2s for 18K samples)
        epochs_per_dimension = []
        for i in range(self._ndim):
            idx_i = data_dims == i
            epochs_i = data_epochs[idx_i]
            epochs_per_dimension.append(epochs_i)
        
        integral_value = 0.0
        
        for m in range(self._ndim):
            integral_value += self._lambda[m] * (end_t - start_t) / 1000.0
            for n in range(self._ndim):
                beta_mn = self._beta[m,n]
                alpha_mn = self._alpha[m,n]
                epochs_n = epochs_per_dimension[n]
                
                epochs_n_ = epochs_n[epochs_n < start_t]
                term1 = np.sum(np.exp(-alpha_mn * (start_t - epochs_n_) / 1000.0) -
                               np.exp(-alpha_mn * (end_t - epochs_n_) / 1000.0))
                
                epochs_n_ = epochs_n[(epochs_n >= start_t) & (epochs_n < end_t)]
                term2 = np.sum(1 - np.exp(-alpha_mn * (end_t - epochs_n_) / 1000.0))
                
                integral_value += beta_mn/alpha_mn * (term1 + term2)
        
        return integral_value

In [39]:
class TimeVaryingMultivariateHawkesProcess:
    _ndim = None # no. of dimensions
    _lambda = None # ndim x 1 vector
    _alpha = None # ndim x ndim matrix
    _beta = None # ndim x ndim matrix
    _mu = None # ndim x nfeatures matrix
    _observation_window = None
    _EPS = 10**-9 # numbers below this are set to 0

    def __init__(self, ndim=None, lam=None, alpha=None, beta=None, mu=None, observation_window=None):
        self._ndim = ndim
        self._lambda = lam
        self._alpha = alpha
        self._beta = beta
        self._mu = mu
        self._observation_window = observation_window
    
    def fit(self, ndim, alpha, F, D, data_epochs, data_dims, observation_window, niter=1000):
        print 'Fitting...'
        print '\tPrecomputing A(i), B(i)...'
        A, B = self._precompute(ndim, alpha, data_epochs, data_dims, observation_window)
                
        print '\tMinimizing negative loglikelihood...'
        alpha_ravel = alpha.ravel()
        nfeatures = D.shape[0]
        x0 = np.random.uniform(low=10**-5.0, high=alpha_ravel[0] - self._EPS, size=(ndim * (ndim + 1) + ndim*nfeatures))
        bounds = [(self._EPS, None) for i in range(ndim)] +\
                 [(self._EPS, None) for i in range(ndim * ndim)] +\
                 [(self._EPS, None) for i in range(ndim * nfeatures)]
                 #[(self._EPS, None) for i in range(ndim * ndim)]
                 
        x, f, d = scipy.optimize.fmin_l_bfgs_b(self._seq_negloglikelihood,
                                               x0=x0, args=(ndim, alpha, data_dims, F, D, observation_window, A, B),
                                               factr=10.0**7.0, iprint=99, pgtol=10**-5, approx_grad=False,
                                               disp=99,
                                               bounds=bounds, callback=self._callback, maxiter=niter, maxfun=niter)

        return x, f, d
    
    @staticmethod
    def _precompute(ndim, alpha, data_epochs, data_dims, observation_window):
        T0, T = observation_window
        nsamples = len(data_epochs)

        A = []
        B = [np.zeros(shape=(ndim, ndim), dtype=np.float64) for k in range(nsamples)]
        for k in range(nsamples):
            # memoize locations of timestamps for each dimension for this sample (2s for 18K samples)
            epochs_per_dimension = []
            for i in range(ndim):
                idx_i = data_dims[k] == i
                epochs_i = data_epochs[k][idx_i]
                epochs_per_dimension.append(epochs_i)
            
            A_k = [np.zeros(shape=(ndim, len(epochs_per_dimension[m])), dtype=np.float64)
                   for m in range(ndim)]
            #B_mn = np.zeros(shape=(ndim, ndim), dtype=np.float64)
            for m in range(ndim):
                epochs_m = epochs_per_dimension[m]
                if len(epochs_m) == 0:
                    continue # no transactions on dimension m
                #A_m = np.zeros(shape=(ndim, len(epochs_m)), dtype=np.float64)
                for n in range(ndim):
                    epochs_n = epochs_per_dimension[n]
                    if len(epochs_n) == 0: # no transactions on dimension n
                        continue
                    len_n = len(epochs_n)
                    A_mn = A_k[m][n] # influence of dimension n on each timestamp of dimension m

                    j = 0
                    
                    if (m != n): # mutual excitation    
                        
                        while j < len_n and epochs_n[j] < epochs_m[0]:
                            j += 1
                            
                        #A_mn[0] = np.sum(np.exp(-alpha[m,n] * (epochs_m[0] - epochs_n[epochs_n < epochs_m[0]]) / 1000.0))
                        A_mn[0] = np.sum(np.exp(-alpha[m,n] * (epochs_m[0] - epochs_n[:j]) / 1000.0))

                        for i in range(1, len(epochs_m)):
                            prev_j = j
                            while j < len_n and epochs_n[j] < epochs_m[i]:
                                j += 1
                            A_mn[i] = np.exp(-alpha[m,n] * (epochs_m[i] - epochs_m[i-1]) / 1000.0) * A_mn[i-1]
                            #A_mn[i] += np.sum(np.exp(-alpha[m,n] *
                            #                         (epochs_m[i] - epochs_n[(epochs_n >= epochs_m[i-1]) &
                            #                                                 (epochs_n < epochs_m[i])]) / 1000.0))
                            A_mn[i] += np.sum(np.exp(-alpha[m,n] *
                                                     (epochs_m[i] - epochs_n[prev_j:j]) / 1000.0))
                        
                        B[k][m,n] = np.sum(1 - np.exp(-alpha[m,n] * (T - epochs_n) / 1000.0))
                    else: # dim_m = dim_n, self-excitation
                        for i in range(1, len(epochs_m)):
                            A_mn[i] = np.exp(-alpha[m,n] * (epochs_m[i] - epochs_m[i-1]) / 1000.0) * (1 + A_mn[i-1])
                        B[k][m,n] = np.sum(1 - np.exp(-alpha[m,n] * (T - epochs_m) / 1000.0))
                
                #A_k.append(A_m)
            
            A.append(A_k)
            #B[k] = B_mn

        return A, B

    def _seq_negloglikelihood(self, params, ndim, alpha, data_dims, F, D, observation_window, A, B):
        nsamples = len(A)
        nfeatures = D.shape[0]
        nll = 0.0
        grad = np.zeros(shape=((ndim + 1) * ndim + ndim*nfeatures), dtype=np.float64) # first row is grad of lambda
        for k in range(nsamples):
            sample_nll, sample_grad = self._negloglikelihood(params, ndim, alpha, data_dims[k],
                                                             F[k], D, observation_window, A[k], B[k])
            nll += sample_nll
            grad += sample_grad
        nll /= nsamples
        grad /= nsamples
        return nll, grad
    
    @staticmethod
    def _negloglikelihood(params, ndim, alpha, data_dims, F, D, observation_window, A, B):
        nfeatures = D.shape[0]
        lam = params[:ndim] # first ndim elements
        beta = params[ndim:ndim+ndim*ndim].reshape((ndim, ndim)) # next ndim * ndim elements
        mu = params[ndim+ndim*ndim:ndim+ndim*ndim + ndim*nfeatures].reshape((ndim,nfeatures))
        T0, T = observation_window
        T_length = (T - T0)/1000.0 # scale down
        
        loglikelihood = 0.0
        grad_lambda = np.zeros(shape=(ndim), dtype=np.float64)
        grad_beta = np.zeros(shape=(ndim, ndim), dtype=np.float64)
        grad_mu = np.zeros(shape=(ndim, nfeatures), dtype=np.float64)

        for m in range(ndim):
            mu_m = mu[m,:]
            dimindex = data_dims == m
            F_m = F[:,dimindex]
            C_m = np.dot(mu_m, F_m)

            Z = lam[m] + np.sum(beta[m,:][:,None] * A[m], axis=0)
            Z += C_m
            loglikelihood += np.sum(np.log(Z)) - np.sum(beta[m,:] * B[m,:] / alpha[m,:]) - lam[m] * T_length
            loglikelihood -= np.sum(mu_m * D)
            grad_lambda[m] = np.sum(1.0/Z) - T_length
            grad_beta[m] = np.sum(A[m]/Z, axis=1) - B[m,:]/alpha[m,:]
            grad_mu[m] = [np.sum(F_m[j]/Z) - D[j] for j in range(nfeatures)]
        
        gradient = np.concatenate((grad_lambda, grad_beta.ravel(), grad_mu.ravel()))
        return -loglikelihood, -gradient

    def check_gradient(self, ndim, alpha, data_epochs, data_dims, F, D, observation_window, delta=10**-8, niter=50):
        print 'Checking gradient...'
        print '\tPrecomputing A, B...'
        A, B = self._precompute(ndim, alpha, data_epochs, data_dims, observation_window)
        
        nfeatures = D.shape[0]
        nsamples = len(data_epochs)
        for i in range(niter):
            row_idx = np.random.randint(0, nsamples)
            a = A[row_idx]
            b = B[row_idx]
            f = F[row_idx]
            d = data_dims[row_idx]
            alpha_ravel = alpha.ravel()
            x0 = np.random.uniform(low=10**-5.0, high=alpha_ravel[0] - self._EPS,
                                   size=(ndim * (ndim + 1) + ndim*nfeatures))
            nll, grad = self._negloglikelihood(x0, ndim, alpha, d, f, D, observation_window, a, b)
            param_idx = np.random.randint(len(x0))
            x0[param_idx] += delta
            new_nll, new_grad = self._negloglikelihood(x0, ndim, alpha, d, f, D, observation_window, a, b)
            num_grad = (new_nll - nll)/delta            
            if abs((grad[param_idx] - num_grad)/num_grad)  > 0.01:
                print 'Gradient checking failed for parameter:', param_idx
                print 'Analytical gradient:', grad[param_idx],
                print 'Numerical gradient:', num_grad
        print '\tGradient checking complete.'
        
    def simulate(self, t0, prev_t, prev_d, horizon_duration):
        simulated_iats = []
        simulated_dims = []
        prev_t_copy = copy.copy(list(prev_t))
        prev_d_copy = copy.copy(list(prev_d))
        prev_t_array = np.array(prev_t_copy, dtype=np.float64)
        prev_d_array = np.array(prev_d_copy, dtype=np.int)
        cumulative_iat = 0.0
        while cumulative_iat < horizon_duration:
            upper_bound = np.sum(self.intensity(t0 + cumulative_iat, (prev_t_array, prev_d_array), include_t0=True))
            s = 1000.0 * np.random.exponential(1.0/upper_bound) # params are in 1/1000s units            
                
            cumulative_iat += s # move ahead in time (may accept or reject)
            if cumulative_iat >= horizon_duration:
                break

            lambda_t_dim = self.intensity(t0 + cumulative_iat, (prev_t_array, prev_d_array))
            lambda_t = np.sum(lambda_t_dim)

            u = np.random.uniform(0.0, 1.0)
            if u <= lambda_t/upper_bound: # accept
                prev_t_copy.append(t0 + cumulative_iat)
                prev_t_array = np.array(prev_t_copy, dtype=np.float64)
                simulated_iats.append(cumulative_iat)
                
                cumprob = np.cumsum(lambda_t_dim/lambda_t)
                u2 = np.random.uniform(0.0, 1.0)
                dim = np.searchsorted(cumprob, u2)
                
                prev_d_copy.append(dim)
                prev_d_array = np.array(prev_d_copy, dtype=np.int)
                simulated_dims.append(dim)

        return simulated_iats, simulated_dims
    
    def predict_dist(self, t0, prev_t, prev_d, niter=1000):
        step = 3600.0 - (t0 % 3600)
        simulated_iats = np.zeros(shape=(niter), dtype=np.float64)
        simulated_dims = np.zeros(shape=(niter), dtype=np.int)
        for i in range(niter):
            cumulative_iat = 0.0
            while True:
                #print 'trial start'
                upper_bound = np.sum(self.intensity(t0 + cumulative_iat, (prev_t, prev_d), include_t0=True))
                s = 1000.0 * np.random.exponential(1.0/upper_bound) # params are in 1/1000s units

                #print 'Candidate next IAT:', t + s
                if s >= step: # upper bound no longer valid
                    cumulative_iat += step
                    step = 3600.0 # one hour in seconds
                    #print '\tLarger than step, moved to:', t
                    continue
                
                cumulative_iat += s # move ahead in time (may accept or reject)
                step -= s
                lambda_t_dim = self.intensity(t0 + cumulative_iat, (prev_t, prev_d))
                lambda_t = np.sum(lambda_t_dim)
                u = np.random.uniform(0.0, 1.0)
                
                if u <= lambda_t/upper_bound: # accept
                    simulated_iats[i] = cumulative_iat
                    if self._ndim > 1:
                        #print 'simulating dim'
                        simulated_dims[i] = np.random.choice(range(self._ndim), p=lambda_t_dim/lambda_t)
                        #print 'done simulating dim'
                    break
    
        return simulated_iats, simulated_dims

    def intensity(self, t0, prev_e, include_t0=False):
        prev_t, prev_d = prev_e
        if not include_t0:
            prev_t_ = prev_t[prev_t < t0]
            prev_d_ = prev_d[prev_t < t0]
        else:
            prev_t_ = prev_t[prev_t <= t0]
            prev_d_ = prev_d[prev_t <= t0]
        
        t0_minus_ti = (t0 - prev_t_) / 1000.0
        intensity = self._lambda + np.sum(self._beta[:, prev_d_] * np.exp(-self._alpha[:, prev_d_] * t0_minus_ti), axis=1)
        
        t0_ = time.gmtime(t0)
        hour_of_day, day_of_week, day_of_month = map(int, time.strftime("%H %w %d", t0_).split(' '))
        day_of_week = (day_of_week - 1) % 7
        
        f = np.array([int(hour_of_day == h) for h in range(24)] +
                     [int(day_of_week >= 0 and day_of_week <= 3)] +
                     [int(day_of_week == 4)] +
                     [int(day_of_week >= 5 and day_of_week <= 6)] +
                     [int(day_of_month == 1)], dtype=np.int)

        #print intensity.shape, self._mu.shape, f.shape, (self._mu * f).shape, np.sum(self._mu * f, axis=0).shape
        intensity += np.sum(self._mu * f, axis=1)
        
        return intensity
    
    def intensity_integral(self, start_t, end_t, data_epochs, data_dims):
        # memoize locations of timestamps for each dimension for this sample (2s for 18K samples)
        epochs_per_dimension = []
        for i in range(self._ndim):
            idx_i = data_dims == i
            epochs_i = data_epochs[idx_i]
            epochs_per_dimension.append(epochs_i)
        
        integral_value = 0.0
        
        for m in range(self._ndim):
            integral_value += self._lambda[m] * (end_t - start_t) / 1000.0
            for n in range(self._ndim):
                beta_mn = self._beta[m,n]
                alpha_mn = self._alpha[m,n]
                epochs_n = epochs_per_dimension[n]
                
                epochs_n_ = epochs_n[epochs_n < start_t]
                term1 = np.sum(np.exp(-alpha_mn * (start_t - epochs_n_) / 1000.0) -
                               np.exp(-alpha_mn * (end_t - epochs_n_) / 1000.0))
                
                epochs_n_ = epochs_n[(epochs_n >= start_t) & (epochs_n < end_t)]
                term2 = np.sum(1 - np.exp(-alpha_mn * (end_t - epochs_n_) / 1000.0))
                
                integral_value += beta_mn/alpha_mn * (term1 + term2)
        
        # mu term integral
        d = np.zeros(28, dtype=np.float64)
        start_timestamp = pd.to_datetime(start_t, unit='s')
        end_timestamp = pd.to_datetime(end_t, unit='s')

        #print 't_i-1', start_timestamp, 't_i', end_timestamp
        
        start_ts_roundup = start_timestamp.floor(freq='H')
        start_seconds = (start_timestamp - start_ts_roundup).total_seconds()
        start_hour = start_timestamp.hour
        end_ts_roundwn = end_timestamp.ceil(freq='H')
        end_seconds = (end_ts_roundwn - end_timestamp).total_seconds()
        end_hour = end_timestamp.hour

        #print 't_i-1', start_timestamp, 't_i', end_timestamp
        
        #hour_periods = pd.date_range(start_timestamp, end_timestamp, freq='s', closed='left')
        hour_periods2 = pd.date_range(start_ts_roundup, end_ts_roundwn, freq='H', closed='left')
        #print hour_periods
        #hour_counts = Counter(hour_periods.hour)
        hour_counts2 = Counter(hour_periods2.hour)
        hour_counts2[start_hour] -= start_seconds/3600.0
        hour_counts2[end_hour] -= end_seconds/3600.0
        #print hour_counts2
        #print start_timestamp, start_ts_roundup, end_timestamp, end_ts_roundwn
        for h in range(24):
            #d[h] = hour_counts[h] / 1000.0
            d[h] = (hour_counts2[h] * 3600.0) / 1000.0
            #print h, hour_counts[h], hour_counts2[h] * 3600.0
            #assert abs(d[h] - (hour_counts2[h] * 3600.0) / 1000.0) < 10**-9
        
        start_ts_roundup = start_timestamp.floor(freq='D')
        start_seconds = (start_timestamp - start_ts_roundup).total_seconds()
        start_day = start_timestamp.weekday()
        end_ts_roundwn = end_timestamp.ceil(freq='D')
        end_seconds = (end_ts_roundwn - end_timestamp).total_seconds()
        end_day = end_timestamp.weekday()
        
        #print start_timestamp, start_ts_roundup, start_seconds/3600.0, end_timestamp, end_ts_roundwn, end_seconds/3600.0
        #day_counts = Counter(hour_periods.weekday)
        day_periods = pd.date_range(start_ts_roundup, end_ts_roundwn, freq='D', closed='left')
        day_counts2 = Counter(day_periods.weekday)
        day_counts2[start_day] -= start_seconds/86400.0
        day_counts2[end_day] -= end_seconds/86400.0
        #for d_i in range(7):
        #    assert abs(day_counts[d_i]/86400.0 - day_counts2[d_i]) < 10**-9
        d[24] = (day_counts2[0] + day_counts2[1] + day_counts2[2] + day_counts2[3]) * 86400.0 / 1000.0
        d[25] = day_counts2[4] * 86400.0/ 1000.0
        d[26] = (day_counts2[5] + day_counts2[6]) * 86400.0 / 1000.0
        
        num_month_starts = np.sum(day_periods.is_month_start)
        #print num_month_starts
        if start_timestamp.is_month_start:
            num_month_starts -= start_seconds/86400.0
        if end_timestamp.is_month_start:
            num_month_starts -= end_seconds/86400.0
        #print start_timestamp, end_timestamp, np.sum(hour_periods.is_month_start), num_month_starts*86400.0
        #assert abs(np.sum(hour_periods.is_month_start) - num_month_starts*86400.0) < 10**-9
        #d[27] = np.sum(hour_periods.is_month_start) / 1000.0
        d[27] = num_month_starts * 86400.0 / 1000.0
        d = np.array(d)
        #print d
        
        mu_term = np.sum(self._mu * d)
        integral_value += mu_term
        
        return integral_value

# Test Models

## Hawkes Process: Simulate Data

In [31]:
simulated_epochs = []
simulated_dims = []
observation_window = [0.0, 600.0]
t0 = observation_window[0]
ndim = 2
true_lambda = 1000.0 * np.array([0.1, 0.1])
true_alpha = 1000.0 * np.array([[1.0, 1.0], [1.0, 1.0]])
true_beta = 1000.0 * np.array([[0.5, 0.01], [0.9, 0.4]])
true_params = np.concatenate((true_lambda, true_beta.ravel()))
print "Generating 100 data samples from a " + str(ndim) + "-D Hawkes process..."
for k in range(100):
    iats, dims = SimpleMultivariateHawkesProcess(ndim=ndim, lam=true_lambda, alpha=true_alpha, beta=true_beta,
                                                 observation_window=observation_window).simulate(t0,
                                                                                                 np.array([]),
                                                                                                 np.array([]),
                                                                                                 observation_window[-1])
    epochs = np.array(iats, dtype=np.float64)
    dims = np.array(dims, dtype=np.int)
    simulated_epochs.append(epochs)
    simulated_dims.append(dims)
print "Done."

Generating 100 data samples from a 2-D Hawkes process...
Done.


## Hawkes Process: Fit Model

Note that alpha is not learned from the data.

In [32]:
est_params, est_nll, d = SimpleMultivariateHawkesProcess().fit(ndim, true_alpha, simulated_epochs,
                                                               simulated_dims, observation_window, niter=1000)
print "Negative loglikelihood:", est_nll
print "Estimated params:", est_params/1000.0
print "True params:", true_params/1000.0
print "Termination info:"
print "\t", d["task"]
print "\t", "Iterations:", d["nit"], "Funcalls:", d["funcalls"]

Fitting...
	Precomputing A(i), B(i)...
	Minimizing negative loglikelihood...
Negative loglikelihood: -2325.514811743731
Estimated params: [0.10101926 0.10131509 0.52093352 0.00443929 0.88401316 0.40952672]
True params: [0.1  0.1  0.5  0.01 0.9  0.4 ]
Termination info:
	CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
	Iterations: 19 Funcalls: 22


## Time-Varying Hawkes Process: Simulate Data

In [62]:
simulated_epochs = []
simulated_dims = []
observation_window = [0.0, 60.0]
t0 = observation_window[0]
ndim = 2
true_lambda = 1000.0 * np.array([0.1, 0.1])
true_alpha = 1000.0 * np.array([[1.0, 1.0], [1.0, 1.0]])
true_beta = 1000.0 * np.array([[0.5, 0.01], [0.9, 0.4]])
true_mu = np.array([[0.2]*6 + [2.0]*12 + [0.2]*6 + [0.1, 2.0, 1.5, 0.0],
                    [0.2]*6 + [0.4]*12 + [2.0]*6 + [2.0, 0.1, 1.0, 0.0]])
true_params = np.concatenate((true_lambda, true_beta.ravel(), true_mu.ravel()))
print "Generating 100 data samples from a " + str(ndim) + "-D Time-Varying Hawkes process..."
for k in range(100):
    iats, dims = TimeVaryingMultivariateHawkesProcess(ndim=ndim, lam=true_lambda, alpha=true_alpha, beta=true_beta,
                                                      mu=true_mu,
                                                      observation_window=observation_window).simulate(t0,
                                                                                                      np.array([]),
                                                                                                      np.array([]),
                                                                                                      observation_window[-1])
    epochs = np.array(iats, dtype=np.float64)
    dims = np.array(dims, dtype=np.int)
    simulated_epochs.append(epochs)
    simulated_dims.append(dims)
print "Done."

Generating 100 data samples from a 2-D Time-Varying Hawkes process...
Done.


## Time-Varying Hawkes Process: Fit Model

In [63]:
# TODO: Precompute D, F

In [ ]:
est_params, est_nll, d = TimeVaryingMultivariateHawkesProcess().fit(ndim, true_alpha, simulated_epochs,
                                                                    simulated_dims, observation_window, niter=1000)
print "Negative loglikelihood:", est_nll
print "Estimated params:", est_params/1000.0
print "True params:", true_params/1000.0
print "Termination info:"
print "\t", d["task"]
print "\t", "Iterations:", d["nit"], "Funcalls:", d["funcalls"]